In [34]:
# to save data from Vedomosti RSS to xml file to the same directory as the script placed
import csv
import requests
import xml.etree.ElementTree as ET
import datetime
import time
import os


In [35]:
#date and time for file name creation
datetime = datetime.datetime.now().strftime('%d%m%Y%H%M%S') 
datetime

'30122022134755'

In [36]:
#saving xml file with data from Lenta RSS
response = requests.get("https://tass.ru/rss/v2.xml")
with open(f'tass{datetime}.xml', "w", encoding="utf-8") as f:
    f.write(response.text)

In [37]:
#to add data from xml file to filesdata table 
import glob
import psycopg2

#the latest file name 
list_of_files = glob.glob('*.xml') 
latest_file = max(list_of_files, key=os.path.getmtime)
print(latest_file)

tass30122022134755.xml


In [38]:
#data from xml file
with open(latest_file, encoding='utf8') as f:
    contents = f.read()
    print(contents)

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0" xmlns:atom="https://www.w3.org/2005/Atom"><channel><title>ТАСС</title>
<description>ИНФОРМАЦИОННОЕ АГЕНТСТВО РОССИИ ТАСС</description>
<language>ru-ru</language>
<link><![CDATA[https://tass.ru]]></link>
<copyright>ТАСС</copyright>
<image><url><![CDATA[https://tass.ru/i/rss/logo.png]]></url>
<title>ТАСС</title>
<link><![CDATA[https://tass.ru]]></link>
</image>
<atom:link href="https://tass.ru/rss/v2.xml" rel="self" type="application/rss+xml"></atom:link>
<item><title>Путин поздравил Лукашенко с Новым годом</title>
<link><![CDATA[https://tass.ru/politika/16721955]]></link>
<guid><![CDATA[https://tass.ru/politika/16721955]]></guid>
<pubDate>Fri, 30 Dec 2022 13:44:58 +0300</pubDate>
<category>Политика</category>
</item>
<item><title>Из Москвы отправился железнодорожный туристический круиз &quot;Крымский вояж&quot;</title>
<link><![CDATA[https://tass.ru/obschestvo/16721869]]></link>
<guid><![CDATA[https://tass.ru/obschestvo/16721869]

In [39]:
#insert data from xml file to filesdata table
try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = """ INSERT INTO filesdata (filename, xmldata) VALUES (%s,%s)"""
    data_to_insert = (f'{latest_file}', f'{contents}')
    cursor.execute(postgres_insert, data_to_insert)

    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

1 record inserted successfully
PostgreSQL connection is closed


In [40]:
#insert categories

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = """ WITH n AS (SELECT distinct text(UNNEST(xpath('//item/category/text()', xmldata))) AS category
FROM filesdata WHERE filename like 'tass%')
INSERT INTO categories (categoryname)
SELECT category FROM n
LEFT JOIN categories ON n.category=categories.categoryname
WHERE category_id is NULL;"""

    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

4 record inserted successfully
PostgreSQL connection is closed


In [41]:
#insert news

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = f"""WITH q AS (WITH w AS (WITH n AS (SELECT RTRIM(LTRIM(text(UNNEST(xpath('//item/title/text()', xmldata))), '<![CDATA['), ']]>') AS title,
	   RTRIM(LTRIM(text(UNNEST(xpath('//item/link/text()', xmldata))), '<![CDATA['), ']]>') AS link,
	   RTRIM(LTRIM(text(UNNEST(xpath('//item/guid/text()', xmldata))), '<![CDATA['), ']]>') AS guid,
	   cast(text(UNNEST(xpath('//item/pubDate/text()', xmldata))) AS timestamp) AS pubDate,
	   text(UNNEST(xpath('//item/description/text()', xmldata))) AS description,
	   text(UNNEST(xpath('//item/enclosure/@url', xmldata))) AS enclosure,
	   RTRIM(LTRIM(text(UNNEST(xpath('//item/category[1]', xmldata))), '<category>'), '</category>') AS category1
	   					  
FROM filesdata
WHERE filename='{latest_file}')
SELECT n.*, c.category_id FROM n
JOIN categories c ON n.category1 = c.categoryname)
SELECT w.* FROM w --include unique rows
LEFT JOIN news nn ON w.guid = nn.guid
WHERE nn.guid IS NULL)
	INSERT INTO news ( title, link, guid, pubdate, description, author, enclosure, pdalink, source_id, file_id)
SELECT  q.title, q.link, q.guid, q.pubdate, q.description, NULL, q.enclosure, NULL, '3',
(SELECT file_id FROM filesdata WHERE filename = '{latest_file}') 
FROM q;"""

 
    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

35 record inserted successfully
PostgreSQL connection is closed


In [42]:
#insert newscategories

try:
    connection = psycopg2.connect(user="postgres",
                                  password="postgres",
                                  host="localhost",
                                  port="5432",
                                  database="news1")
    cursor = connection.cursor()

    postgres_insert = f"""INSERT INTO newscategories (news_id, category_id)
WITH a AS (WITH o AS (WITH l AS (SELECT RTRIM(LTRIM(text(UNNEST(xpath('//item/guid/text()', xmldata))), '<![CDATA['), ']]>') AS guid,
	   RTRIM(LTRIM(text(UNNEST(xpath('//item/category[1]', xmldata))), '<category>'), '</category>') AS category
FROM filesdata
WHERE filename='{latest_file}')
SELECT n.news_id, n.guid, l.category
FROM news n
JOIN l ON n.guid = l.guid)
SELECT o.*, c.category_id FROM o
JOIN categories c ON o.category = c.categoryname)
SELECT a.news_id, a.category_id FROM a 
LEFT JOIN newscategories nc ON a.news_id = nc.news_id
WHERE nc.news_id IS NULL;"""
   
    cursor.execute(postgres_insert)
    
    connection.commit()
    count = cursor.rowcount
    print(count, "record inserted successfully")

except (Exception, psycopg2.Error) as error:
    print("failed to insert", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

35 record inserted successfully
PostgreSQL connection is closed
